## Ejecutar las siguientes líneas

In [6]:
import sys
from os import path, mkdir, listdir, chdir
import pickle
import cv2 as cv
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import matplotlib.pyplot as plt
from skimage import io
import pandas as pd
import numpy as np

modulos_path = path.abspath('../minIA')
if modulos_path not in sys.path:
    sys.path.append(modulos_path)
    
from utiles import lectura_img
import random
random.seed(42)



In [7]:
#Recibe lista de keypoints de una imagen, crea los objetos KP de openCV
def genKeyPoints( kp_img ):
    keypoints = list()
    for kp in kp_img:
        keypoints.append(cv.KeyPoint(kp[0], kp[1], kp[2]))
    return keypoints

## Visualización tópicos

El siguiente codigo carga los tópicos y elementos de la figura para mostrar los archivos

In [8]:
@interact_manual
def load_topicos(
    topics_path=[(file,path.join("/media/working/minia/minhash/",file)) for file in listdir("/media/working/minia/minhash/") if file.endswith(".models")],
    centXimg='/media/working/minia/descriptores/labels_full_dataset_v2_SIFT.pickle',
    descr_kp='/media/working/minia/descriptores/full_dataset_v2_SIFT.pickle', 
    image_dir='/media/working/minia/images/images_training_rev1/'):
    global topics
    global len_topics
    global images_descr
    global desc_kp
    global etiquetas
    global dir_out
    global dir_img
    global topics_lenght
    
    topics = open(topics_path, 'r')
    topics= topics.readlines()
    topics_lenght=[]
    for line in topics:
        centroides = line.strip().split()[1:]
        centroides = [ int(cent.split(':')[0]) for cent in centroides]
        topics_lenght.append(len(centroides))
        
    len_topics=len(topics)
    
    pickle_file = open(centXimg,'rb')
    images_descr = pickle.load(pickle_file)
    pickle_file.close()
    
    pickle_file = open(path.abspath(descr_kp), 'rb')  
    args = pickle.load(pickle_file)
    desc_kp = pickle.load(pickle_file)
    pickle_file.close()
    
    dir_img=image_dir

    print('#Topics: '+ str(len_topics))

interactive(children=(Dropdown(description='topics_path', options=(('index_inv_full_dataset_v2_SIFT.r4l10000o0…

In [45]:
def indicesTopicos (centroides, images_descr,threshold=0.7):
    tam_cents = len(centroides)
    centroides_= set(centroides) 
    img_index = 0
    images = list()
    for descrp in images_descr[0]:
        descp_ = dict([(ic,i) for i,ic in enumerate(descrp)])
        descp = set(descrp)
        tam_descp = len(descrp)
        inter=centroides_.intersection(descp)
        overlap=len(inter)/min(len(centroides_),len(descp))
        if overlap >= threshold:
            images.append((img_index,[ descp_[ic] for ic in inter],overlap))
        #Overlaping
        #max_ = round( min(tam_descp, tam_cents)*.30 ) #Requiere un 30%
        #posc_index = 0
        #posc = list()
        #for cent in descrp:
        #    if cent in centroides:
        #        posc.append(posc_index)
        #    posc_index += 1
        #if len(posc) > max_ : #No puede ser >= porque max_ puede ser cero
        #    images.append((img_index ,posc)) #Añade una tupla, del indice de la imagen y los indices de los KP
        img_index += 1
    return images

## Elegir tópico

Este código permite elejir el tópico, el número de imágnes dónde buscar el código y el threshold a usar para considerar que un tópico esta presente o no en la imagen (topicos con números pequeños usar 1.0)

In [46]:
@interact_manual
def choose_topicos(Topics=[(f"{i} [size {z}]",i) for i,z in zip(range(len_topics),topics_lenght)], num_imgs= (0, int(len(images_descr)/1), 5), threshold = (0.0, 1.0, 0.01)):
    global lista_imgs
    global num_topic
    global name_images
    
    num_topic= Topics
    centroides = topics[Topics].split()[1:]
    centroides = [ int(cent.split(':')[0]) for cent in centroides]
    lista_imgs = indicesTopicos (centroides, images_descr[0:num_imgs], threshold=threshold)
    
    imgs_index=[imgs[0] for imgs in lista_imgs]
    aux=0
    name_images=[]
    for ii,i in enumerate(imgs_index):
        imagen = path.abspath(dir_img + desc_kp[i]['name_img'])
        name_images.append((desc_kp[i]['name_img'],aux,i,lista_imgs[ii][2]))
        aux+=1
        
    print('#Images: '+str(len(lista_imgs)))
    print('#Len topic: '+str(len(centroides)))

interactive(children=(Dropdown(description='Topics', options=(('0 [size 49]', 0), ('1 [size 376]', 1), ('2 [si…

## Visualizando ejemplos del tópico

Se muestran ejemplos de imágenes que contiene el tópico seleccionado en la parte [anterior](http://minhashing.ngrok.io/notebooks/minIA/notebooks/keyPoints.ipynb#Elegir-t%C3%B3pico)

In [55]:
plt.rcParams["figure.figsize"] = (10,5)
plt.rcParams['figure.dpi'] = 160

@interact
def show_images_per_topic(img=[(aux,(img,aux,i,o)) for img,aux,i,o in name_images]):
    img,aux,i,o=img
    if aux-5 < 0:
        aux=5
    if aux+5 > len(name_images):
        aux=len(name_images)-5
    min_=max(0,aux-5)
    max_=min(len(name_images),aux+5)
    fig, axs = plt.subplots(2,5)
    for ii in range(min_,max_):
        img,aux,i,overlap=name_images[ii]
        imagen = path.abspath(dir_img + img)
        img = cv.imread(imagen)
        gray= cv.cvtColor(img,cv.COLOR_BGR2GRAY)
        key_points= [desc_kp[i]['keypoints'][kp] for kp in lista_imgs[aux][1]]
        keypoints = genKeyPoints(key_points)
        img=cv.drawKeypoints(gray,keypoints,img,color=(255,0,0),flags=cv.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
        x=ii-min_
        y=x%5
        x=int(x/5)
        axs[x,y].set_yticklabels([])
        axs[x,y].set_xticklabels([])
        axs[x,y].set_xlabel(f'{ii}, {overlap:0.3}, #{i}')
        axs[x,y].imshow(img)

interactive(children=(Dropdown(description='img', options=((0, ('165129.jpg', 0, 580, 0.68)), (1, ('418071.jpg…

## Ver una imágen

In [52]:
@interact_manual
def choose_topicos(image=str(random.choice(range(1,len(images_descr)))),threshold = (0.0, 1.0, 0.01),text=True):
    global lista_imgs
    global topics
    global name_images
    global images_descr
    cmap=plt.get_cmap("hsv")

    descrp=images_descr.iloc[int(image)][0]
    img=images_descr.iloc[int(image)][1]
    descp = set(descrp)
    tam_descp = len(descrp)
    topics_in_image= 0
    
    imagen = path.abspath(dir_img + img)
    img = cv.imread(imagen)
    gray= cv.cvtColor(img,cv.COLOR_BGR2GRAY)
    gray= cv.cvtColor(gray,cv.COLOR_GRAY2BGR)
    keypoints=[]
    for topic in range(len(topics)):
        color=tuple(int(c*256) for c in cmap(topic/len(topics))[:3])
        centroides = topics[topic].split()[1:]
        centroides = set([ int(cent.split(':')[0]) for cent in centroides])
        
        descp_ = dict([(ic,i) for i,ic in enumerate(descrp)])
        inter=centroides.intersection(descp)
        overlap=len(inter)/min(len(centroides),len(centroides))
        
        if overlap >= threshold:
            key_points= [desc_kp[int(image)]['keypoints'][kp] for kp in [ descp_[ic] for ic in inter]]
            keypoints_ = genKeyPoints(key_points)
            topics_in_image+=1
            for curKey in keypoints_:
                x=np.int(curKey.pt[0])
                y=np.int(curKey.pt[1])
                size = np.int(curKey.size)
                gray=cv.circle(gray,(x,y),size,color=color,thickness=1, lineType=0, shift=0)
                if text:
                    gray=cv.putText(gray,str(topic),(x,y),cv.FONT_HERSHEY_SIMPLEX ,0.3,color,1, cv.LINE_AA)
            
    plt.imshow(gray, cmap=cmap)
    print(topics_in_image)
    

interactive(children=(Text(value='1640', description='image'), FloatSlider(value=0.5, description='threshold',…